In [93]:
import pandas as pd
import numpy as np
import datetime
import re
import os

In [104]:
DTYPE={

    0:  str,      # RecNum                ## str in original schema
    1:  int,      # FiscalYear            ## str in original schema
    2:  str,      # entity name
    3:  str,      # Fund1
    4:  str,      # Fund2
    5:  str,      # Fund3
    6:  str,      # Fund4
    7:  str,      # Org1
    8:  str,      # Org2
    9:  str,      # Org3
    10: str,      # Org4
    11: str,      # Org5
    12: str,      # Org6
    13: str,      # Org7
    14: str,      # Org8
    15: str,      # Org9
    16: str,      # Org10
    17: str,      # Trans Type
    18: str,      # Cat1                  ## conditional
    19: str,      # Cat2
    20: str,      # Cat3
    21: str,      # Cat4
    22: str,      # Cat5
    23: str,      # Cat6
    24: str,      # Cat7
    25: str,      # PCV name              ## conditional
    26: str,      # DBA
    27: str,      # vndr Id
#   28: str,      # posting date
    29: str,      # Trans Desc
    30: str,      # transaction ID
    31: str,      # ref id
    32: str,      # contact name
    33: str,      # contract num
    34: str,      # title
    35: str, # hourly rate           ## float take NAs
    36: str,      # gender
    37: str, # amount
    38: str,      # masked
    39: str,      # UCA
            }


#path_str = "H:/Transparency/Transparent Utah 3.0/Raw Batches/rawBatch/_data_56125.txt"
path_str = "H:/Transparency/Transparent Utah 3.0/Raw Batches/rawBatch/_data_63533.txt"
batch_num =re.search("\d{5}(?=.txt$)",path_str).group()
batch_name  = 'batch_'+batch_num 



df=pd.read_csv(path_str,
               header=None, sep='|',
               thousands=',',
               low_memory=False,
               parse_dates=[28],
               encoding='cp1252',
               dtype=DTYPE
              )




info = [
    'RcNm', 'FsYr', 'Name',
    'Fnd1', 'Fnd2', 'Fnd3', 'Fnd4',
    'Org1', 'Org2', 'Org3', 'Org4', 'Org5', 'Org6', 'Org7', 'Org8', 'Org9', 'OrgX',
    'Type',
    'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7',
    'Vndr', 'VDBA', 'VIDC', 'Pdat', 'Desc', 'TID', 'TRef', 'Cnam', 'Cnum', 'PTit', 'Hrat', 'Gend', 'Amnt', 'PI',
]
uca = [ 'UCA', ]
psd = [
    'Prg1', 'Prg2', 'Prg3', 'Prg4', 'Prg5', 'Prg6', 'Prg7',
    'Fnc1', 'Fnc2', 'Fnc3', 'Fnc4', 'Fnc5', 'Fnc6', 'Fnc7',
]
COLLEN = len(df.columns)

if COLLEN == 40:
    info += uca
elif COLLEN == 54:
    info += uca + psd
    PSD = True

elif COLLEN ==41:
    info += uca + ['EMPTY']

elif COLLEN ==55:
    info += uca + psd + ['EMPTY']
    PSD = True
    
    
df.rename(inplace=True, columns=lambda x: info[x])

ParserError: Error tokenizing data. C error: Expected 43 fields in line 5, saw 49


In [95]:
# create a few statistic df
stat_columns = ['entity_name', 'batch_num', 'error_type']
df_valid = pd.DataFrame(columns=stat_columns)
df_Hrat = pd.DataFrame(columns=stat_columns)
df_Amnt = pd.DataFrame(columns=stat_columns)
df_Gend = pd.DataFrame(columns=stat_columns)

In [96]:
df.head()

,RcNm,FsYr,Name,Fnd1,Fnd2,Fnd3,Fnd4,Org1,Org2,Org3,...,Desc,TID,TRef,Cnam,Cnum,PTit,Hrat,Gend,Amnt,PI
0,1,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180001,NaN,NaN,NaN,District Manager,,M,69004.78,
1,2,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180002,NaN,NaN,NaN,Secretary,24.82,F,21560.38,
2,3,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180003,NaN,NaN,NaN,Administrative Assistant,15.32,F,884.73,
3,4,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180004,NaN,NaN,NaN,Administrative Assistant,14.00,F,2932.02,
4,5,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180005,NaN,NaN,NaN,Operator,24.82,M,3134.77,


In [97]:
ent_name = df.iloc[1,2]

In [103]:
#Find the hourly rates, and if it is an " " string, replace it with an empty string ""  
#Once flagged, we convert it to a 0. 

if len(df[df['Hrat'] == ' '].index) > 0 :
    
    print(ent_name + " with "+ batch_name+' has Hrat with \' \' entries. Correcting them and converting to numeric')    
    
    #write to global df
    df_Hrat.loc[len(df_Hrat)] = [ent_name, batch_num, 'entries with \' \'']
    
    #replace the empty string and convert to numeric.
    df.loc[df.Hrat == ' ', 'Hrat'] = ''
    df['Hrat'] = df['Hrat'].str.replace(',','')
    df['Hrat'] = pd.to_numeric(df['Hrat'],errors='coerce')
    
else:
    print(batch_name+' Converting to numeric')
    df['Hrat'] = df['Hrat'].str.replace(',','')
    df['Hrat'] = pd.to_numeric(df['Hrat'],errors='coerce')
    
    #now check that the the Transaction type = PY
    if len( df[df['Type'] != 'PY' & df['Hrat'].notnull()] ) > 0 : 
        
        print(ent_name + " with "+ batch_name+' has Hrat entires without trans Type = PY. Flag and write to error df')    
        
        #write to global df
        df_Hrat.loc[len(df_Hrat)] = [ent_name, batch_num, 'transaction type != PY']
    else:
        print(batch_name+' has Hrat with no errors. writing info to success df')
        df_valid.loc[len(df_valid)] = [ent_name, batch_num, 'pass']
           

batch_56125 Converting to numeric


AttributeError: Can only use .str accessor with string values!

In [88]:
df_Hrat
# Woohoo it works!

,entity_name,batch_num,error_type
0,Mountain Green Sewer Improvement District,56125,entries with ' '


In [89]:
#Find the amount, and if it is an " " string, replace it with an empty string ""  
#Once flagged, we convert it to a 0. 



if len(df[df['Amnt'] == ' '].index) > 0 :
    
    print(ent_name + " with "+ batch_name+' has Amnt with \' \' entries. Correcting them and converting to numeric')    
    df_Hrat.loc[len(df_Hrat)] = [ent_name, batch_num, 'entries with \' \'']
    
    df.loc[df.Hrat == ' ', 'Amnt'] = ''
    df['Amnt'] = df['Amnt'].str.replace(',','')
    df['Amnt'] = pd.to_numeric(df['Amnt'],errors='coerce')
else:
    print(batch_name+' has Amnt with no errors. Converting to numeric')
    df['Amnt'] = pd.to_numeric(df['Amnt'],errors='coerce')

batch_56125 has Amnt with no errors. Converting to numeric


In [80]:
df_Amnt # should be empty since there were no errors. 

,entity_name,batch_num,error


In [47]:
#Validate the Gender field. 
if len(df[df['Gend'] == ' '].index) > 0 :
    print(batch_name+' has Gend containing unacceptable character. Correcting them and converting to numeric')
    df.loc[df.Gend == ' ', 'Gend'] = ''
    df['Gend'] = df['Gend'].str.replace(',','')
    df['Gend'] = pd.to_numeric(df['Gend'],errors='coerce')
else:
    print(batch_name+' has Gend with no errors.')
    df['Gend'] = pd.to_numeric(df['Gend'],errors='coerce')

,RcNm,FsYr,Name,Fnd1,Fnd2,Fnd3,Fnd4,Org1,Org2,Org3,...,Desc,TID,TRef,Cnam,Cnum,PTit,Hrat,Gend,Amnt,PI
0,1,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180001,NaN,NaN,NaN,District Manager,NaN,M,69004.78,
1,2,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180002,NaN,NaN,NaN,Secretary,24.82,F,21560.38,
2,3,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180003,NaN,NaN,NaN,Administrative Assistant,15.32,F,884.73,
3,4,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180004,NaN,NaN,NaN,Administrative Assistant,14.00,F,2932.02,
4,5,2018,Mountain Green Sewer Improvement District,Enterprise,NaN,NaN,NaN,District,,NaN,...,Regular Wages,20180005,NaN,NaN,NaN,Operator,24.82,M,3134.77,
